In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/allotment.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know. don't make it up : \n\n {context}"),
        ("human", "{question}"),
    ]
)

chain = {"context" : retriever, "question" : RunnablePassthrough()} | prompt | llm

chain.invoke("What is the allotment meaning?")



Created a chunk of size 1384, which is longer than the specified 600
Created a chunk of size 1450, which is longer than the specified 600
Created a chunk of size 1485, which is longer than the specified 600
Created a chunk of size 1364, which is longer than the specified 600
Created a chunk of size 1243, which is longer than the specified 600
Created a chunk of size 1317, which is longer than the specified 600
Created a chunk of size 1434, which is longer than the specified 600
Created a chunk of size 1367, which is longer than the specified 600
Created a chunk of size 1448, which is longer than the specified 600
Created a chunk of size 1391, which is longer than the specified 600
Created a chunk of size 1412, which is longer than the specified 600
Created a chunk of size 1406, which is longer than the specified 600
Created a chunk of size 1482, which is longer than the specified 600
Created a chunk of size 1322, which is longer than the specified 600
Created a chunk of size 1350, whic

AIMessage(content='An allotment is a plot of land made available for individual, non-commercial gardening for growing food plants, forming a kitchen garden away from the residence of the user.')

In [6]:
vectorstore.similarity_search("what is the meaning of allotment")

[Document(page_content='Allotment (gardening)\rAllotments in Germany\rAn\xa0allotment\xa0(British English),[1]\xa0is a plot of land made available for individual, non-commercial gardening for growing food plants, so forming a\xa0kitchen garden\xa0away from the residence of the user. Such plots are formed by subdividing a piece of land into a few or up to several hundred parcels that are assigned to individuals or families, contrary to a\xa0community garden\xa0where the entire area is tended collectively by a group of people. [2]\xa0The term "victory garden" is also still sometimes used, especially when a garden dates back to the\xa0First\xa0or\xa0Second World War. The individual size of a parcel typically suits the needs of a family, and often the plots include a shed for tools and shelter, and sometimes a hut for seasonal or weekend accommodation. The individual gardeners are usually organised in an allotment association, which leases or is granted the land from an owner who may be a 